In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cervical_Cancer"
cohort = "GSE131027"

# Input paths
in_trait_dir = "../../input/GEO/Cervical_Cancer"
in_cohort_dir = "../../input/GEO/Cervical_Cancer/GSE131027"

# Output paths
out_data_file = "../../output/preprocess/Cervical_Cancer/GSE131027.csv"
out_gene_data_file = "../../output/preprocess/Cervical_Cancer/gene_data/GSE131027.csv"
out_clinical_data_file = "../../output/preprocess/Cervical_Cancer/clinical_data/GSE131027.csv"
json_path = "../../output/preprocess/Cervical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"High frequency of pathogenic germline variants in genes associated with homologous recombination repair in patients with advanced solid cancers"
!Series_summary	"We identified pathogenic and likely pathogenic variants in 17.8% of the patients within a wide range of cancer types. In particular, mesothelioma, ovarian cancer, cervical cancer, urothelial cancer, and cancer of unknown primary origin displayed high frequencies of pathogenic variants. In total, 22 BRCA1 and BRCA2 germline variant were identified in 12 different cancer types, of which 10 (45%) variants were not previously identified in these patients. Pathogenic germline variants were predominantly found in DNA repair pathways; approximately half of the variants were within genes involved in homologous recombination repair. Loss of heterozygosity and somatic second hits were identified in several of these genes, supporting possible causality for cancer development. A potential treatment t

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Dict, Any, Callable, Optional

# 1. Gene Expression Data Availability
# Based on the data, we see gene-related information suggesting gene expression data might be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Identify keys for trait, age, and gender
# For trait, we can use cancer type (key 1)
trait_row = 1

# Age and gender information are not present in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value (cancer type) to binary for Cervical Cancer."""
    if not isinstance(value, str):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value is Cervical cancer
    if value.lower() == 'cervical cancer':
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age value to continuous type."""
    # Not applicable as age data is not available
    return None

def convert_gender(value):
    """Convert gender value to binary (0: female, 1: male)."""
    # Not applicable as gender data is not available
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None
if trait_row is not None:
    try:
        # Get the clinical data from the previous step's variable
        # We assume clinical_df is available from a previous step
        # Or rebuild it from the sample characteristics dictionary if needed
        # Creating a simple clinical dataframe from the sample characteristics provided
        # This assumes the sample characteristics were presented in a format similar to what we need
        sample_chars = {0: ['tissue: tumor biopsy'], 
                      1: ['cancer: Breast cancer', 'cancer: Colorectal cancer', 'cancer: Bile duct cancer', 
                          'cancer: Mesothelioma', 'cancer: Urothelial cancer', 'cancer: Pancreatic cancer', 
                          'cancer: Melanoma', 'cancer: Hepatocellular carcinoma', 'cancer: Ovarian cancer', 
                          'cancer: Cervical cancer', 'cancer: Head and Neck cancer', 'cancer: Sarcoma', 
                          'cancer: Prostate cancer', 'cancer: Adenoid cystic carcinoma', 'cancer: NSCLC', 
                          'cancer: Oesophageal cancer', 'cancer: Thymoma', 'cancer: Others', 'cancer: CUP', 
                          'cancer: Renal cell carcinoma', 'cancer: Gastric cancer', 
                          'cancer: Neuroendocrine cancer', 'cancer: vulvovaginal']}
        
        # Create a clinical dataframe in the expected format
        clinical_data = pd.DataFrame()
        for key, values in sample_chars.items():
            for value in values:
                sample_id = f"Sample_{len(clinical_data) + 1}"
                clinical_data.loc[sample_id, key] = value
        
        # Extract clinical features
        selected_clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_features)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the clinical data to the specified output file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        print("Clinical data processing skipped.")


Preview of selected clinical features:
{0: [nan], 1: [0.0]}
Clinical data saved to ../../output/preprocess/Cervical_Cancer/clinical_data/GSE131027.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs from a microarray platform
# They follow the format like "1007_s_at", "1053_at" which are typical Affymetrix IDs
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# They need to be mapped to human gene symbols for downstream analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns for gene identifiers and gene symbols
# Based on the preview, 'ID' stores the probe identifiers and 'Gene Symbol' stores gene symbols
probe_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Use get_gene_mapping to extract these columns and create a mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# Print a small preview of the mapping to verify its structure
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the resulting gene expression data
print(f"\nGene expression data shape after mapping: {gene_data.shape}")

# Print the first few gene symbols
print("\nFirst few gene symbols after mapping:")
print(gene_data.index[:10])


Gene mapping preview (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A

Gene expression data shape after mapping: (21278, 92)

First few gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Debug the clinical data to understand its structure better
print("Original clinical data shape:", clinical_data.shape)
print("Clinical data index:", clinical_data.index)
print("Clinical data columns:", clinical_data.columns)

# Reconstruct clinical features - Create a dictionary of sample IDs to their cancer types
cancer_types = {}
for sample_id, row in clinical_data.iterrows():
    for col in clinical_data.columns:
        if col != '!Sample_geo_accession':
            value = row.get(col)
            if isinstance(value, str) and 'cancer:' in value:
                cancer_type = value.split(':', 1)[1].strip()
                is_cervical = 1 if cancer_type.lower() == 'cervical cancer' else 0
                cancer_types[sample_id] = is_cervical

# Create a DataFrame for the trait (Cervical_Cancer)
trait_df = pd.DataFrame({trait: cancer_types})
clinical_features_df = trait_df.T  # Transpose to match required format

print("Clinical features DataFrame shape:", clinical_features_df.shape)
print("Clinical features DataFrame (first 5 rows):")
print(clinical_features_df.head())

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data using the library function
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print("Linked data shape after handling missing values:", linked_data.shape)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data for various cancer types including cervical cancer."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Cervical_Cancer/gene_data/GSE131027.csv
Original clinical data shape: (24, 2)
Clinical data index: Index(['Sample_1', 'Sample_2', 'Sample_3', 'Sample_4', 'Sample_5', 'Sample_6',
       'Sample_7', 'Sample_8', 'Sample_9', 'Sample_10', 'Sample_11',
       'Sample_12', 'Sample_13', 'Sample_14', 'Sample_15', 'Sample_16',
       'Sample_17', 'Sample_18', 'Sample_19', 'Sample_20', 'Sample_21',
       'Sample_22', 'Sample_23', 'Sample_24'],
      dtype='object')
Clinical data columns: Index([0, 1], dtype='int64')
Clinical features DataFrame shape: (1, 23)
Clinical features DataFrame (first 5 rows):
                 Sample_2  Sample_3  Sample_4  Sample_5  Sample_6  Sample_7  \
Cervical_Cancer         0         0         0         0         0         0   

                 Sample_8  Sample_9  Sample_10  Sample_11  ...  Sample_15  \
Cervical_Cancer         0         0          0          1  ...          0   

                 Sample_16  Sampl